In [1]:
include("FMfcns_RWC11282017.jl")
include("StressInvUtils_RWC_09142017.jl")

starting_direction (generic function with 1 method)

<p> Specify Strike, Dip, and Rake </p>

In [2]:
strike = 0*pi/180;
dip = 90*pi/180;
rake = 0*pi/180;

strike_aux = 270*pi/180
dip_aux = 90*pi/180
rake_aux = 180*pi/180

3.141592653589793

<p> Calculate moment tensor and normal/slip vectors </p>

In [3]:
M = moment_tensor(strike, dip, rake)
P, N, T = moment_tensor_2_PTN(M);
normal, normal_aux, slip, slip_aux = PTN_2_fault_geometry(P, T, N);
println(P)
println(N)
println(T)
#print(slip_aux)

[0.707107,-0.707107,0.0]
[0.0,0.0,1.0]
[-0.707107,-0.707107,0.0]


<p> Check to see if Fault Rotation Matrix is working </p>

In [4]:
Rf = FaultOrientation_Matrix(strike, dip);
Rf_aux = FaultOrientation_Matrix(strike_aux, dip_aux)
normal_fault = Rf'*normal;
normal_fault_aux = Rf_aux'*normal_aux;

slip_fault = Rf'*slip;
slip_fault_aux = Rf_aux'*slip_aux;

println("Fault Normals")
println(normal_fault)
println(normal_fault_aux)

println("Fault Slips")
println(slip_fault)
println(slip_fault_aux)



Fault Normals
[-1.83009e-16,-2.22045e-16,1.0]
[6.50354e-17,-2.22045e-16,1.0]
Fault Slips
[-1.0,3.48631e-32,-1.57009e-16]
[1.0,1.44408e-32,-6.50354e-17]


<p> Estimate Orientations of P, T, and N from Moment tensor </p>


In [5]:
rotm = [P N T]
angles = rotation_to_euler(rotm)

3×2 Array{Float64,2}:
  1.5708    -1.5708 
 -0.0        3.14159
 -0.785398   2.35619

In [6]:
rotation_phi = rotation_matrix([0 0 -1], angles[3, 1])
rotation_theta = rotation_matrix([0 1 0 ], angles[2, 1])
mcs_prime = rotation_theta*rotation_phi*[1; 0; 0];
rotation_rho = rotation_matrix(mcs_prime, -angles[1, 1]);
R = rotation_rho*rotation_theta*rotation_phi;
pstress = [-1 0 0; 0 -1/2 0; 0 0 -1/4];
stress_geo = R*pstress*R';
s1, s2, s3 = moment_tensor_2_PTN(stress_geo);
println(s1)
println(P)
println(s2)
println(N)
println(s3)
println(T)


[0.707107,-0.404107,0.0]
[0.707107,-0.707107,0.0]
[0.0,0.0,1.0]
[0.0,0.0,1.0]
[-0.707107,-0.707107,0.0]
[-0.707107,-0.707107,0.0]


In [14]:
n_pred, n_pred_aux, s_pred, s_pred_aux = PTN_2_fault_geometry(s1, s3, s2)
println(s_pred)

[-1.0,2.35514e-16,0.0]


In [8]:
println(s1, P)
println(s2, N)
println(s3, T)

[0.707107,-0.707107,0.0][0.707107,-0.707107,0.0]
[0.0,0.0,1.0][0.0,0.0,1.0]
[-0.707107,-0.707107,0.0][-0.707107,-0.707107,0.0]


In [38]:
stress_fault = Rf'*stress_geo*Rf
println("Stress on Fault")
stress_fault

Stress on Fault


3×3 Array{Float64,2}:
 -0.625         7.51373e-17  -0.375      
  7.51373e-17  -0.5           6.36406e-17
 -0.375         6.36406e-17  -0.625      

In [48]:
atan2(stress_fault[2, 3], stress_fault[1, 3])-pi

0.0

In [47]:
m_rot = Rf'*M*Rf
atan2(m_rot[2, 3], m_rot[1, 3])-pi

0.0

-0.3750000000000001